In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use("fivethirtyeight")
from IPython.display import Image
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, classification_report, roc_auc_score, roc_curve
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.datasets import make_classification;

/Users/georgemcintire/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# <b>Logistic Regression</b>

- Logistic regression is a generalization of the linear regression model adapted to
 to classification problems.
 
- Very popular because it's very fast and interpretable. Doesn't need scaling or much tuning.

- Not vulnerable to overfitting when you don't have many features.
 
- In linear regression, we use a set of quantitative feature variables to predict a continuous response variable. In logistic regression, we use a set of quantitative feature variables to predict probabilities of class membership.

- Named for the function used at the core of the method, the logistic function aka the sigmoid function. 

- Logistic regression is a linear regression between our feature, X, and the log-odds of our data belonging to a certain class that we will call true for the sake of generalization.

In [ ]:
Image("logit_model.png")
#Source: Sinan Ozdemir

The preceding graph represents the logistic function's ability to map our continuous input, x, to a smooth probability curve that begins at the left, near probability 0, and as we increase x, our probability of belonging to a certain class rises naturally and smoothly up to probability 1. 


In other words:

    • Logistic regression gives an output of the probabilities of a specific class being true
    
    • Those probabilities can be converted into class predictions: if p>= 0.5 the models returns 1 and if p<.0.5 it returns 0
    
    • Logistic function is S-shaped and will always produced values > 0 and < 1.
    
    • Not all relationships as you know are linear, so LR is not always the right model.
    


### <b>Key difference in use of coefficients in linear vs logistic</b>
<br>
Linear Regression: Betas/coefficients represents the change in the response variable for a unit change in x. 

Logistic Regression: They represents the change in the log-odds. For a unit change in x. This means that e^β gives us the change in the odds for a unit change in x.

Coding time:

In [ ]:
#Use Sklearn to create fake data
data = make_classification(n_samples=300,n_features=2,class_sep=.86,n_informative=2,
                         n_redundant=0, n_repeated=0,n_classes=2, random_state=8)
#returns a tuple with the features in index 0 and target in index 1

In [ ]:
#Transform fake data into pandas dataframe
cols = ["feature_one", "feature_two"]
df = pd.DataFrame()
df["target"] = 
#assign red to class 0 and blue to class 1. For plotting purposes.
color = 

Let's visualize the fake data and try to imagine what a linear boundary separating the two groups would look like.

In [ ]:
plt.figure(figsize=(15,11))
plt.scatter(df.feature_one, df.feature_two, c=color, s=80)
plt.xlabel("Feature One",)
plt.ylabel("Feature Two");

If you had to draw a straight line that best separates the two classes, where would you put the line?
<br><br>
Let's focus on Feature One and plot it against the target variable

In [2]:
plt.figure(figsize=(12,8))
plt.scatter(df.feature_one, df.target, s=60)
plt.xlabel("Feature One")
plt.ylabel("Target");

Whats the relationship between the two variables?

In [3]:
#Sort the dataframe by the feature one and create a new data frame from that.
df2 = df.sort("feature_one").copy();

Create a linear regression model trained on feature one to predict the target variable

In [ ]:
#Create X and y variables
X = df2[["feature_one"]]
y = df2.target
#Intialize model

#Fit model

#Make predictions on X and assign them to pred variable
;

Plot the data with the predictions

In [4]:
plt.figure(figsize=(12,8))
plt.xlabel("Feature One")
plt.ylabel("Target")
plt.scatter(X,y, s=60)
plt.plot(X, pred, c="r", linewidth=3);


<b> How does this model look?</b>

<b> Look at the S-function from above, how do you think it would fit the data? </b>

<br><br><br><br><br><br><br>


<b> Let's find out!</b>

Let's fit a logistic regression model on the data above and plot the predicted labels and the probabilities

In [ ]:
#Intialize and fit a logistic regression model
logr = LogisticRegression();

In [ ]:
#Score the model 
score = 
print "The accuracy score is {:.2f} percent".format(score*100)

In [ ]:
#Generate label predictions


In [ ]:
#Assign label predictions to pred_labels
pred_labels = 

In [ ]:
#Generate probability predictions



The first column indicates the predicted probability of class 0, and the second column indicates the predicted probability of class 1.

In [5]:
#Assign probability of class 1 to pred_probs


### <b>Put it all together and have a gander</b>

In [6]:
plt.figure(figsize=(18,12))
plt.xlabel("Feature One")
plt.ylabel("Target")

plt.scatter(X,y, s=70, c= "blue", alpha=1, label="Scatter Plot Data")
plt.plot(X, pred, c="r", linewidth=8, alpha=.5, label = "Linear Regression Predictions")
plt.plot(X, pred_labels, c="y", linewidth=8, alpha=.5, label = "Logistic Regression Label Predictions")
plt.plot(X, pred_probs, c="g", linewidth=8, alpha=.5, label = "Logistic Regression Probability Predictions")

plt.legend(loc=4, fontsize="xx-large");

### <b> What do you see? What catches your attention?</b>

Go back to the original dataset with two features and visualize the linear boundary

In [7]:
#This function allows us to see the model
def plot_decision_boundary(model, X, y):
    X_max = X.max(axis=0)
    X_min = X.min(axis=0)
    xticks = np.linspace(X_min[0], X_max[0], 100)
    yticks = np.linspace(X_min[1], X_max[1], 100)
    xx, yy = np.meshgrid(xticks, yticks)
    ZZ = model.predict(np.c_[xx.ravel(), yy.ravel()])
    Z = ZZ >= 0.5
    Z = Z.reshape(xx.shape)
    fig, ax = plt.subplots()
    ax = plt.gca()
    ax.contourf(xx, yy, Z, cmap=plt.cm.bwr, alpha=0.2)
    ax.scatter(X[:,0], X[:,1], c=y,s=40, alpha=0.4)
    plt.xlabel("Feature One")
    plt.ylabel("Feature Two")

In [ ]:
#Create X and y variables from data using df
X = df.drop("target", axis=1)
y = df.target
#Color code y
color = y.map({0:"blue", 1:"red"})

In [ ]:
#Intialize model and fit it to X and y
;

In [8]:
#Pass the model and the data into the plot_decision_boundary function
# plot_decision_boundary(lr, X.values,color);

<b> Does the boundary above match the one I asked you to imagine earlier? </b>

## <b> Can you use Spotify data to predict whether or not I will like a song? </b>

<b> Attributes </b>


    Acousticness: A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
    
    Danceability: Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.

    Instrumentalness: Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
    
    Loudness: The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typical range between -60 and 0 db.
    
    Mode: Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.

    Valence: A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry).
    
    Tempo: The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.

    Energy: Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
    
More information here https://developer.spotify.com/web-api/get-audio-features/

Dataset of songs I like and dislike. Target == 1 means liked song and Target == 0 means disliked song

In [ ]:
#Load the datafile "Spotify_Data.pkl" and check it out
df = pd.read_pickle("Spotify_Data.pkl")
df.head()

<b>Quick EDA</b>: Summary stats grouped by class and correlations

In [ ]:
#What is the average value for each feature grouped by class?

In [9]:
#What are the correlations of the features?

<b> Thoughts? Things of interest? Which variables stick out to you? </b>

### <b> Train a logistic regression model on the data to predict whether or not I will like a certain song </b>

In [ ]:
#Create X and y variables
X = 
y = 

#Intialize, fit, and score the model
lr = 

lr.fit()

score = lr.score()

print ("The model produces an accuracy score of {:.2f} percent".format(score*100))

Is that a good or bad score? To find out let's compare it to the null accuracy.

In [ ]:
#Find the null accuracy aka the benchmark score. Percents for each class.


### <b> Training/testing  </b>
1. Split the data into train/test splits
2. Fit data onto training set
3. Make predictions on test set with the training model
4. Calculate accuracy score by comparing predicted labels of the test set to its actual labels

In [ ]:
#Step 1. Use test_size = 0.4 and random_state = 42
X_train, X_test, y_train, y_test = train_test_split()

#Step 2


#Step 3


#Step 4
testing_score = accuracy_score()

print ("The model accurately classified {:.2f} percent of the testing data".format(testing_score*100))

How does the testing accuracy compare to the first one?
<br><br><br><br>
Use cross validation to derive a truer testing accuracy score

In [ ]:
#Use cross_val_score method to generate the average accuracy score for 5 CVs
mean_cv_score = cross_val_score()

print ("The cross validated accuracy score is {:.2f} percent").format(mean_cv_score*100)

### <b>Probability, odds, e, log, log-odds</b>
<br>
Quick stats and probability detour

In [ ]:
Image("odds_probs.png")

In [ ]:
# create a table of probability versus odds
table = pd.DataFrame({'probability':[0.1, 0.2, 0.25, 0.5, 0.6, 0.8, 0.9]})
table['odds'] = table.probability/(1 - table.probability)
table

What is e? It is the base rate of growth shared by all continually growing processes:

In [ ]:
# exponential function: e^1
np.exp(1)

What is a (natural) log? It gives you the time needed to reach a certain level of growth:

In [ ]:
# time needed to grow 1 unit to 2.718 units
np.log(2.718)

It is also the inverse of the exponential function:

In [ ]:
np.log(np.exp(5))

In [ ]:
# add log-odds to the table
table['logodds'] = np.log(table.odds)
table

The log odds are what is passed throught the logistic function

<b>Train model using one feature: "valence"</b>

In [ ]:
#V is the variable assigned to valence

V = 
lr_V = 


In [ ]:
# compute predicted log-odds for valence_value=0.5...
#by multiplying it by coefficien and then adding the intercept to it

valence_value = 0.5
logodds = lr_V.intercept_ + lr_V.coef_[0]*valence_value
logodds

In [ ]:
# convert log-odds to odds
odds = np.exp(logodds)
odds

In [ ]:
# convert odds to probability
prob = odds/(1 + odds)
prob

In [ ]:
# compute predicted probability for valence_value using the predict_proba method
lr_V.predict_proba(valence_value)[:, 1]

Positive coefficients increase the log-odds of the response (and thus increase the probability), and negative coefficients decrease the log-odds of the response (and thus decrease the probability).
<br><br><br><br><br><br><br><br>
Re-fit model but all the variables and make table of the coefficients and odds

In [ ]:
X = df.drop("target", axis=1)
y = df.target
lr = LogisticRegression()
lr.fit(X,y);

In [ ]:
#Table of coefficients and their values
coef = pd.DataFrame(zip(X.columns, np.transpose(lr.coef_[0])), columns=["coef", "value"])
coef

Odds ratio is the ratio of the odds(after increasing X_i by 1) over (divided) by odds(before increasing X_i by 1).

In [ ]:
coef_odds = np.e**(coef["value"])
coef["odds_ratio"] = coef_odds
coef

The increase in probability is hard to quantify.  The lower p(before) is, the greater increase you'll have vs. a higher p(before).

In [ ]:
#Visualize how the Beta and intercept can affect the probabilities
Image("curves.png")
#Source: Kevin Markham

Changing the $\beta_0$ or intercept value shifts the curve horizontally, whereas changing the $\beta_1$ or coefficient value changes the slope of the curve.

## <b> AUC and ROC Curves </b>

ROC = Receiver operating characteristic
<br>
AUC = Area under curve

### <b>Recall and Precision scores</b>
<br><br>
<b>Recall aka the True Positive Rate:</b> What percent of the data that is labelled class A was correctly identified as class A 
<br><br>
<b>Precision:</b> What percent of the data that is classified as class A actually belongs to class A

In [ ]:
Image("acc_rec_prec.png")

In [ ]:
#Visualize example confusion matrix
Image("con_matrix.png")
#Source: Kevin Markham

### <b>Create confusion matrix for the Spotify data and calculate recall and precision scores

In [ ]:
#Make a train test split of the spotify data and train a logistic regression model
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.4,
                                                    random_state=4)

preds = 

In [ ]:
#Null accuracy of y_test


In [ ]:
#Pass the predictions and y_test into a confusion matrix
confusion_matrix(y_test, preds)

In [ ]:
#Calculate accuracy, precision and recall scores
acs = float(accuracy_score(,))
ps = float(precision_score(,))
rs = float(recall_score(,))

print ("Accuracy score = {:.2f}".format(acs*100))
print ("Precision score = {:.2f}".format(ps*100))
print ("Recall score = {:.2f}".format(rs*100))

<b>False Positive Score:</b> Number of events wrongly classified as True divided by the number of negative events:
<br><br>
Example: Giving a bank loan to someone who defaults

In [ ]:
#We have to calculate false positive score using our function

def FPR(YT, PR): 
    """
    YT is the target variable for the testing set
    PR is the predictions made on the features of the testing set
    """
    CM = confusion_matrix(YT, PR)
    #Fill in blank
    return FPR

In [ ]:
#Call FPR on the y_test and predicted labels objects
FPR()

Onto plotting the ROC curve

In [ ]:
#Derive probabilities of class 1 from the test set
test_probs = lr.predict_proba(X_test)[:,1]
#Pass in the test_probs variable and the true test labels aka y_test in the roc_curve function
fpr, tpr, thres = roc_curve(y_test, test_probs)
#Will explain the fpr, tpr, thresholds

In [10]:
#Plotting False Positive Rates vs the True Positive Rates
#Dotted line represents a useless model
plt.figure(figsize=(15,11))
plt.plot(fpr, tpr, linewidth=8)
plt.plot([0,1], [0,1], "--", alpha=.7)
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")

plt.show()

<b> What does this tell us about the model? Is it a good or bad model?<b>

In [ ]:
#Caculate the area under the curve score using roc_auc_score
roc_auc_score(y_test, test_probs)

Take a look at some other roc curves

In [ ]:
Image("roc_curves.png",width=500, height=500)

<b> How thresholds affect the TPR and FPR? </b>

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(thres, fpr, linewidth=5)
plt.xlabel("Thresholds")
plt.ylabel("False Positive Rate");

In [ ]:
plt.figure(figsize=(12,9))
plt.plot(thres, tpr, linewidth=5)
plt.xlabel("Thresholds")
plt.ylabel("True Positive Rate");

What do you see here? Why are is there a negative correlation in both graphs?

### <b>Thresholds and model performance: Does tweaking the threshold give us a better model?</b>
<br><br>
Unfortunately there's no threshold to configure in a logistic regression model. 
<br>For example: lr = logisticregression(threshold=n) 
<br>
<br>So we need to create our own threshold function using the np.where method

In [ ]:
#Assign all the values in test_probs >=0.7 == 1 and the rest equal to 0
#First argument is condition
#Second argument is the value you use to replace all the values that satisfy the condition
#Third argument is the value you use to replace all the values that don't satisfy the condition
labels_70 = np.where(test_probs>=0.7, 1, 0)
labels_70[:20]

Does this give a better accuracy score?

In [ ]:
# accuracy_score(y_test, labels_70)

In [ ]:
#Put this in function form

def thres_acc(T, YT, TP):
    """
    T == Threshold input of our choosing
    YT == The target variable for the testing set
    TP == Probabilities of Class 1 for the features of the testing set
    """
    
    return 

### <b>Plot various thresholds vs their accuracy scores</b>

In [ ]:
#Create an array of 20 values evenly spaced between 0 and 1
thresholds = np.linspace(0,1, 30).tolist()
acc_scores = [thres_acc(i, y_test, test_probs) for i in thresholds]

In [ ]:
#Plot thresholds vs accuracy scores
plt.figure(figsize=(15,11))
plt.plot(thresholds, acc_scores, linewidth=5)
plt.xlabel("Thresholds")
plt.ylabel("Accuracy Scores")
plt.show()

Which threshold produces the best accuracy score?

In [12]:
#Make dictornary of thresholds and acc_scores.
thres_score_dict = 


In [ ]:
#Sort dictionary to find key with the highest value


# <b>In class work:</b>

    - Use feature engineering to create the best possible model to predict songs I like.
    - Use ROC curve to evaluate your models.
    - See which features are the most useful in predicting song category
    - Experiment with .predict_proba(). Try to find the which values of the features produces the highest probability.  